In [68]:
from configPy import Config
from pathlib import Path
from PIL import Image
from src.aux_imgs import get_resolution, get_resolution_folder
import numpy as np

In [2]:
files_dir = Config.get_dir_files()
print(files_dir)

/home/henrique/Documents/projects/teste/tratar_dados_av3/files


In [3]:
imgs_dir = files_dir["imgs"]
data_label_dir = files_dir["data_label"]

In [4]:
imagens = list(imgs_dir.list_files().values())
print(imagens[:5])
labels = list(data_label_dir.list_files().values())
print(labels[:5])

[PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/imgs/1.jpg'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/imgs/8.jpg'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/imgs/16.jpg'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/imgs/9.jpg'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/imgs/4.jpg')]
[PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/data_label/11.txt'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/data_label/12.txt'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/data_label/9.txt'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/data_label/18.txt'), PosixPath('/home/henrique/Documents/projects/teste/tratar_dados_av3/files/data/data_label/15.txt')]


## Funções

In [ ]:
# renomear arquivos de label
def rename_file(file: Path):
    origin_name = file.name
    if 'xml' not in origin_name:
        return
    new_name = ".".join([element for element in origin_name.split(".") if element != "xml"])
    new_name_path = file.parent / new_name
    file.rename(new_name_path)

In [5]:
# funcao improvisada para pegar o par de imagem e seu label
def get_pair(num: int):
    return f"{num}.jpg", f"{num}.txt"

In [17]:
# partir do path, pegar label
def get_label(label: Path):
    with open(label, "r") as f:
        linhas = f.readlines()
    labels = []
    for line in linhas:
        line = line.split(" ")
        labels.append({"x":float(line[1]),"y":float(line[2]), "w":float(line[3]), "h":float(line[4])})
    return labels


## Coisas

In [6]:
img, label = get_pair(10)
img_path = imgs_dir[img]
label_path = data_label_dir[label]

In [18]:
w, h = get_resolution(img_path)
label = get_label(label_path)
print(w, h)
print(label)

1600 980
[{'x': 0.285, 'y': 0.120408, 'w': 0.0425, 'h': 0.081633}, {'x': 0.454062, 'y': 0.176531, 'w': 0.013125, 'h': 0.036735}]


In [44]:
# associar cada imagem as suas lables e dimençõs

data_ = {}

for i in range(1, 20):
    img, label = get_pair(i)
    # dimencoes da imagem
    dim = get_resolution(imgs_dir[img])
    data_[i] = (dim, get_label(data_label_dir[label]))

In [50]:
# média de proporção entre a janela e a imagem
mean_w_window = mean_h_window = 0
qunt_window = 0

mean_h = mean_w = 0
qunt = 0
for key, value in data_.items():
    img_size, list_dict_window = value

    mean_w += img_size[0]
    mean_h += img_size[1]
    qunt += 1
    for dict_window in list_dict_window:
        mean_w_window += dict_window['w']
        mean_h_window += dict_window['h']
        qunt_window += 1
mean_w_window, mean_h_window = mean_w_window/ qunt_window, mean_h_window/ qunt_window
mean_w, mean_h = mean_w/ qunt, mean_h/ qunt 

In [52]:
print("Media das imgs (tamanho real): ",mean_w, mean_h)
print("Media das janelas : ", mean_w_window, mean_h_window)

Media das imgs (tamanho real):  1845.3684210526317 1328.0
Media das janelas :  0.027447772727272727 0.05298099999999999


In [42]:
# trazer as janelas para as proporções certas

data_prop = {}

for i in range(1, len(data_) + 1):

    w, h = data_[i][0]
    list_dict_window = data_[i][1]
    new_list = []
    for dict_window in list_dict_window:
        new_dict = {}
        new_dict['x'] = dict_window['x']*w
        
        new_dict['y'] = dict_window['y']*h

        new_dict['w'] = dict_window['w']*w

        new_dict['h'] = dict_window['h']*h
        new_list.append(new_dict)
    data_prop[i] = ((w, h), new_list)
    

In [54]:
# achar proporção entre tamanho da janela e imagem

# média de proporção entre a janela e a imagem
# média de proporção entre a janela e a imagem
mean_w_window = mean_h_window = 0
qunt_window = 0

mean_h = mean_w = 0
qunt = 0
for key, value in data_prop.items():
    img_size, list_dict_window = value

    mean_w += img_size[0]
    mean_h += img_size[1]
    qunt += 1
    for dict_window in list_dict_window:
        mean_w_window += dict_window['w']
        mean_h_window += dict_window['h']
        qunt_window += 1
mean_w_window, mean_h_window = mean_w_window/ qunt_window, mean_h_window/ qunt_window
mean_w, mean_h = mean_w/ qunt, mean_h/ qunt 

In [55]:
print("Media das imgs (tamanho real): ",mean_w, mean_h)
print("Media das janelas : ", mean_w_window, mean_h_window)

Media das imgs (tamanho real):  1845.3684210526317 1328.0
Media das janelas :  51.045349045454536 71.95452159090908


In [60]:
# lista das caixas

lista_caixas = []

for value in data_prop.values():
    _, list_dict_window = value

    for dict_window in list_dict_window:
        lista_caixas.append((dict_window["w"], dict_window["h"]))

In [61]:
for i in range(len(lista_caixas)):
    lista_caixas[i] = lista_caixas[i][0] * lista_caixas[i][1]

In [66]:
list_sizes = [128**2, 256**2, 512**2, 1024**2]

In [70]:
for num in list_sizes:
    print(f"para a área {num}: {np.mean(lista_caixas)/num}")

para a área 16384: 0.26454266365908824
para a área 65536: 0.06613566591477206
para a área 262144: 0.016533916478693015
para a área 1048576: 0.004133479119673254
